Instalación de librerías y pre-requisitos:

In [1]:
import pandas as pd
from prettytable import PrettyTable


Carga de datos en DataFrames:

In [2]:
df_indices = pd.read_csv("indices.csv", sep=";")
df_estructuras = pd.read_csv("estructuras.csv", sep=";")
df_resultados = pd.read_csv("resultados.csv", sep=";")


Preprocesamiento de los DataFrames:

In [3]:
dic_headers = {}

dic_headers['df_indices'] = df_indices.columns.tolist()
dic_headers['df_estructuras'] = df_estructuras.columns.tolist()
dic_headers['df_resultados'] = df_resultados.columns.tolist()


In [4]:
def formatear_a_float(x):
  x = x.str.replace('.', '', regex=False)
  x = x.str.replace(',', '.', regex=False)
  x = x.astype(float)

  return x

for clave in dic_headers:
  header = dic_headers[clave]
  df = globals()[clave]

  for i in range (len(header)):
    nombre_col = header[i]

    df[nombre_col] = df[nombre_col].astype(str)
    df[nombre_col] = df[nombre_col].str.strip()

    match clave:
      case 'df_indices':
        if i >= 3:
          df[nombre_col] = formatear_a_float(df[nombre_col])

      case 'df_estructuras':
        if nombre_col == 'Ponderación':
          df[nombre_col] = df[nombre_col].str.replace('%', '', regex=False)
          df[nombre_col] = df[nombre_col].astype(float)
          df[nombre_col] = df[nombre_col].apply(lambda x: x / 100)

      case 'df_resultados':
        if i != 0:
          df[nombre_col] = formatear_a_float(df[nombre_col])


Procesamiento y análisis:

In [5]:
dic_ponderaciones = {}

for i in range (len(df_estructuras)):
  nombre = df_estructuras.iloc[i, 2]
  ponderacion = df_estructuras.iloc[i, 3]

  dic_ponderaciones[nombre] = ponderacion

print(dic_ponderaciones)


{'Productos lácteos': 0.15, 'Harinas': 0.05, 'Bebidas no alcohólicas': 0.3, 'Productos de carnes, pescados, frutas, legumbres, hortalizas y aceites': 0.5, 'Otros productos textiles': 0.3, 'Tejidos y artículos de punto': 0.1, 'Cuero, artículos de marroquinería y calzado': 0.52, 'Productos textiles': 0.08, 'Papel y productos de papel': 0.8, 'Envases de papel y cartón': 0.18, 'Otros artículos de papel y cartón': 0.02, 'Madera y productos de madera excepto muebles': 0.3, 'Sustancias químicas básicas': 0.25, 'Abonos y fertilizantes': 0.45}


In [6]:
dic_acum23 = {}

for i in range (len(df_estructuras)):
  nombre = df_estructuras.iloc[i, 2]
  num_fila = df_indices.index[df_indices['Nombre'] == nombre].to_numpy()[0]

  acum23: float = 0
  for j in range (4, len(df_indices.columns)):
    acum23 += df_indices.iloc[num_fila, j]

  dic_acum23[nombre] = acum23

print(dic_acum23)


{'Productos lácteos': 31591.77, 'Harinas': 33446.42, 'Bebidas no alcohólicas': 27501.980000000003, 'Productos de carnes, pescados, frutas, legumbres, hortalizas y aceites': 22421.34, 'Otros productos textiles': 26288.65, 'Tejidos y artículos de punto': 28345.95, 'Cuero, artículos de marroquinería y calzado': 30737.340000000004, 'Productos textiles': 33584.10999999999, 'Papel y productos de papel': 41229.700000000004, 'Envases de papel y cartón': 42684.44999999999, 'Otros artículos de papel y cartón': 29414.55, 'Madera y productos de madera excepto muebles': 29773.629999999997, 'Sustancias químicas básicas': 38937.15000000001, 'Abonos y fertilizantes': 31207.96}


In [7]:
df_desglose = pd.DataFrame({
    'Concepto': [],
    'Índice': [],
    'Ponderación': [],
    'Acum 2023': [],
    'Acum 2023 x Ponderación': []
})

for i in range (len(df_estructuras)):
  concepto = df_estructuras.iloc[i, 0]
  indice = df_estructuras.iloc[i, 2]
  ponderacion = dic_ponderaciones[indice]
  acum23 = dic_acum23[indice]
  acum23_ponderado = acum23 * ponderacion

  df_desglose.loc[i] = [concepto, indice, ponderacion, acum23, acum23_ponderado]

df_desglose['Acum 2023'] = df_desglose['Acum 2023'].round(2)
df_desglose['Acum 2023 x Ponderación'] = df_desglose['Acum 2023 x Ponderación'].round(2)

df_desglose = df_desglose.sort_values(by=['Concepto', 'Índice'], ascending=[True, True])
df_desglose = df_desglose.reset_index(drop=True)

#print(df_desglose)


In [8]:
# Agrupar por 'Concepto'
conceptos = df_desglose['Concepto'].unique()

for concepto in conceptos:
    tabla = PrettyTable(['Índice', 'Ponderación', 'Acum 2023', 'Acum 2023 x Ponderación'])

    subgrupo = df_desglose[df_desglose['Concepto'] == concepto]

    for _, row in subgrupo.iterrows():
        tabla.add_row([row['Índice'], row['Ponderación'], row['Acum 2023'], row['Acum 2023 x Ponderación']])

    print(f"\nConcepto: {concepto}")
    print(tabla)



Concepto: Cartoneros
+-----------------------------------+-------------+-----------+-------------------------+
|               Índice              | Ponderación | Acum 2023 | Acum 2023 x Ponderación |
+-----------------------------------+-------------+-----------+-------------------------+
|     Envases de papel y cartón     |     0.18    |  42684.45 |          7683.2         |
| Otros artículos de papel y cartón |     0.02    |  29414.55 |          588.29         |
|     Papel y productos de papel    |     0.8     |  41229.7  |         32983.76        |
+-----------------------------------+-------------+-----------+-------------------------+

Concepto: Comedores
+------------------------------------------------------------------------+-------------+-----------+-------------------------+
|                                 Índice                                 | Ponderación | Acum 2023 | Acum 2023 x Ponderación |
+------------------------------------------------------------------------

In [9]:
df_conceptos = pd.DataFrame({
    'Concepto': [],
    'Participación relativa (%)': [],
    'Porcentaje de ejecución (%)': []
})

base_devengado_total = df_resultados['Devengado mensual'].sum()

for i in range (len(df_resultados)):
  concepto = df_resultados.iloc[i, 0]
  devengado_mensual = df_resultados.iloc[i, 2]
  area_devengar = df_resultados.iloc[i, 3]
  monto_proyectado = df_resultados.iloc[i, 1]

  participacion = devengado_mensual/base_devengado_total * 100
  ejecucion = area_devengar/monto_proyectado * 100

  df_conceptos.loc[i] = [concepto, participacion, ejecucion]

df_conceptos['Participación relativa (%)'] = df_conceptos['Participación relativa (%)'].round(2)
df_conceptos['Porcentaje de ejecución (%)'] = df_conceptos['Porcentaje de ejecución (%)'].round(2)

df_conceptos = df_conceptos.sort_values(by=['Concepto'], ascending=[True])
df_conceptos = df_conceptos.reset_index(drop=True)

#print(df_conceptos)


In [10]:
tabla2 = PrettyTable()

tabla2.field_names = df_conceptos.columns.tolist()

for row in df_conceptos.itertuples(index=False):
    tabla2.add_row(row)

print(tabla2)

+-----------------+----------------------------+-----------------------------+
|     Concepto    | Participación relativa (%) | Porcentaje de ejecución (%) |
+-----------------+----------------------------+-----------------------------+
|    Cartoneros   |            0.15            |            91.57            |
|    Comedores    |            0.65            |            99.89            |
| Espacios Verdes |           96.67            |            97.12            |
|    Vestimenta   |            2.54            |            99.99            |
+-----------------+----------------------------+-----------------------------+
